<a href="https://colab.research.google.com/github/saritmaitra/AL-VAR-VECM-/blob/main/VARMAX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import datetime as dt
import seaborn as sns
%matplotlib inline
from statsmodels.tsa.statespace.varmax import VARMAX
from statsmodels.tsa.stattools import adfuller
from sklearn import metrics
from timeit import default_timer as timer
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from google.colab import files
uploaded = files.upload()

Saving AL_final_data.csv to AL_final_data.csv


In [3]:
# Import Gold data in panda and passed to a variable name "df1"
data = pd.read_csv("AL_final_data.csv")
data.head()

,Unnamed: 0,timestamp,es,eur_usd,gbp_usd,gc,nq,usd_jpy
0,0,2020-05-01 00:00:00,2877.125,1094465.0,1258140.0,1695.85,8846.125,107301000.0
1,1,2020-05-01 00:01:00,2875.375,1094215.0,1257935.0,1695.60,8841.250,107368000.0
2,2,2020-05-01 00:02:00,2874.625,1094165.0,1257765.0,1695.65,8837.750,107376500.0
3,3,2020-05-01 00:03:00,2874.125,1094115.0,1257565.0,1695.40,8836.750,107381500.0
4,4,2020-05-01 00:04:00,2875.250,1094190.0,1257535.0,1694.85,8841.125,107376500.0


In [4]:
data.drop(['Unnamed: 0'], axis=1, inplace=True)
data.tail()

,timestamp,es,eur_usd,gbp_usd,gc,nq,usd_jpy
29340,2020-05-29 20:56:00,3027.25,1111740.0,1235400.0,1731.50,9588.625,107675000.0
29341,2020-05-29 20:57:00,3027.25,1111750.0,1235450.0,1731.20,9588.500,107830000.0
29342,2020-05-29 20:58:00,3027.25,1111650.0,1235450.0,1731.45,9588.625,107830000.0
29343,2020-05-29 20:59:00,3027.25,1113400.0,1235450.0,1731.30,9587.375,107835000.0
29344,2020-05-29 21:00:00,3027.25,1113400.0,1237150.0,1731.35,9591.125,107835000.0


In [5]:
# time-series evaluation function, as shown here:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true,y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true,y_pred)}')

In [7]:
# ADF test function to check for stationary data

def Augmented_Dickey_Fuller_Test_func(series , column_name):
    print (f'Results of Dickey-Fuller Test for column:{column_name}')
    dftest = adfuller(series, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic',
                                             'p-value','No Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
           dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dftest[1] <= 0.05:
           print("Conclusion:====>")
           print("Reject the null hypothesis")
           print("Data is stationary")
    else:
           print("Conclusion:====>")
           print("Fail to reject the null hypothesis")
           print("Data is non-stationary")
           
for name, column in data[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']].iteritems():
           Augmented_Dickey_Fuller_Test_func(data[name],name)
           print('\n')

Results of Dickey-Fuller Test for column:eur_usd
Test Statistic                    -0.131445
p-value                            0.946215
No Lags Used                       8.000000
Number of Observations Used    29336.000000
Critical Value (1%)               -3.430573
Critical Value (5%)               -2.861639
Critical Value (10%)              -2.566822
dtype: float64
Conclusion:====>
Fail to reject the null hypothesis
Data is non-stationary


Results of Dickey-Fuller Test for column:gbp_usd
Test Statistic                    -2.671934
p-value                            0.079022
No Lags Used                       8.000000
Number of Observations Used    29336.000000
Critical Value (1%)               -3.430573
Critical Value (5%)               -2.861639
Critical Value (10%)              -2.566822
dtype: float64
Conclusion:====>
Fail to reject the null hypothesis
Data is non-stationary


Results of Dickey-Fuller Test for column:usd_jpy
Test Statistic                    -1.989365
p-value  

In [8]:
data = data[['es','eur_usd','gbp_usd','gc','nq','usd_jpy']]

In [16]:
nobs = 15
X = data[:15000]
train, test = X[0:-nobs], X[-nobs:]

# Check size
print(train.shape)  
print(test.shape)

(14985, 6)
(15, 6)


In [17]:
# Make the data stationary by using Pandas differencing.
train_diff = train.diff().dropna()

# Check whether the variables are stationary after first differencing.
for name, column in train_diff[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']].iteritems():
    Augmented_Dickey_Fuller_Test_func(train_diff[name],name)
    print('\n')

Results of Dickey-Fuller Test for column:eur_usd
Test Statistic                   -50.535424
p-value                            0.000000
No Lags Used                       7.000000
Number of Observations Used    14976.000000
Critical Value (1%)               -3.430787
Critical Value (5%)               -2.861733
Critical Value (10%)              -2.566873
dtype: float64
Conclusion:====>
Reject the null hypothesis
Data is stationary


Results of Dickey-Fuller Test for column:gbp_usd
Test Statistic                   -49.852516
p-value                            0.000000
No Lags Used                       7.000000
Number of Observations Used    14976.000000
Critical Value (1%)               -3.430787
Critical Value (5%)               -2.861733
Critical Value (10%)              -2.566873
dtype: float64
Conclusion:====>
Reject the null hypothesis
Data is stationary


Results of Dickey-Fuller Test for column:usd_jpy
Test Statistic                   -28.626331
p-value                          

In [11]:
# cointegraion test

from statsmodels.tsa.vector_ar.vecm import coint_johansen
def cointegration_test(df):
    res = coint_johansen(df,-1,5)
    d = {'0.90':0, '0.95':1, '0.99':2}
    traces = res.lr1
    cvts = res.cvt[:, d[str(1-0.05)]]
    def adjust(val, length= 6):
        return str(val).ljust(length)
    print('Column Name > Test Stat > C(95%) => Signif \n','--'*20)
    for col, trace, cvt in zip(df.columns, traces, cvts):
        print(adjust(col), '> ', adjust(round(trace,2), 9), ">",adjust(cvt, 8), ' => ' , trace > cvt)

cointegration_test(train_diff[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']])

Column Name > Test Stat > C(95%) => Signif 
 ----------------------------------------
eur_usd >  16357.01  > 83.9383   =>  True
gbp_usd >  12808.71  > 60.0627   =>  True
usd_jpy >  9682.09   > 40.1749   =>  True
gc     >  7059.77   > 24.2761   =>  True
nq     >  4622.5    > 12.3212   =>  True
es     >  2241.16   > 4.1296    =>  True


In [12]:
# inverse differencing function.
def inverse_diff(actual_df, pred_df):
    df_res = pred_df.copy()
    columns = actual_df.columns
    for col in columns:
        df_res[str(col)+'_1st_inv_diff'] = actual_df[col].iloc[-1] + df_res[str(col)].cumsum()
    return df_res

In [13]:
from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.model_selection import ParameterGrid

"""
Let’s define a parameter grid for selecting p, q, and trend.
"""
param_grid = {'p': [1,2,3], 'q':[1,2,3], 'tr': ['n','c','t','ct']}
pg = list(ParameterGrid(param_grid))
pg

[{'p': 1, 'q': 1, 'tr': 'n'},
 {'p': 1, 'q': 1, 'tr': 'c'},
 {'p': 1, 'q': 1, 'tr': 't'},
 {'p': 1, 'q': 1, 'tr': 'ct'},
 {'p': 1, 'q': 2, 'tr': 'n'},
 {'p': 1, 'q': 2, 'tr': 'c'},
 {'p': 1, 'q': 2, 'tr': 't'},
 {'p': 1, 'q': 2, 'tr': 'ct'},
 {'p': 1, 'q': 3, 'tr': 'n'},
 {'p': 1, 'q': 3, 'tr': 'c'},
 {'p': 1, 'q': 3, 'tr': 't'},
 {'p': 1, 'q': 3, 'tr': 'ct'},
 {'p': 2, 'q': 1, 'tr': 'n'},
 {'p': 2, 'q': 1, 'tr': 'c'},
 {'p': 2, 'q': 1, 'tr': 't'},
 {'p': 2, 'q': 1, 'tr': 'ct'},
 {'p': 2, 'q': 2, 'tr': 'n'},
 {'p': 2, 'q': 2, 'tr': 'c'},
 {'p': 2, 'q': 2, 'tr': 't'},
 {'p': 2, 'q': 2, 'tr': 'ct'},
 {'p': 2, 'q': 3, 'tr': 'n'},
 {'p': 2, 'q': 3, 'tr': 'c'},
 {'p': 2, 'q': 3, 'tr': 't'},
 {'p': 2, 'q': 3, 'tr': 'ct'},
 {'p': 3, 'q': 1, 'tr': 'n'},
 {'p': 3, 'q': 1, 'tr': 'c'},
 {'p': 3, 'q': 1, 'tr': 't'},
 {'p': 3, 'q': 1, 'tr': 'ct'},
 {'p': 3, 'q': 2, 'tr': 'n'},
 {'p': 3, 'q': 2, 'tr': 'c'},
 {'p': 3, 'q': 2, 'tr': 't'},
 {'p': 3, 'q': 2, 'tr': 'ct'},
 {'p': 3, 'q': 3, 'tr': 'n'},
 {

In [ ]:
"""
TIME EXPENSIVE PROCESS

Evaluating all possible combinations of p, q, and trend, and storing the results in a df_results_moni DataFrame for further
reference
"""
df_results_moni = pd.DataFrame(columns=['p', 'q', 'tr','RMSE_EURUSD','RMSE_GBPUSD','RMSE_USDJPY',
                                        'RMSE_GC', 'RMSE_NQ', 'RMSE_ES'])
print('starting grid search')
start = timer()
for a,b in enumerate(pg):
    p = b.get('p')
    q = b.get('q')
    tr = b.get('tr')
    model = VARMAX(train_diff, order=(p,q), trend=tr).fit()
    z = model.forecast(y=train_diff[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']].values, steps=nobs)
    df_pred = pd.DataFrame(z, columns=['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es'])
    res = inverse_diff(data[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']],df_pred)
    eur_usd_rmse = np.sqrt(metrics.mean_squared_error(test.eur_usd, res.eur_usd_1st_inv_diff))
    gbp_usd_rmse = np.sqrt(metrics.mean_squared_error(test.gbp_usd, res.gbp_usd_1st_inv_diff))
    usd_jpy_rmse = np.sqrt(metrics.mean_squared_error(test.usd_jpy, res.usd_jpy_1st_inv_diff))
    gc_rmse = np.sqrt(metrics.mean_squared_error(test.gc, res.gc_1st_inv_diff))
    nq_rmse = np.sqrt(metrics.mean_squared_error(test.nq, res.nq_1st_inv_diff))
    es_rmse = np.sqrt(metrics.mean_squared_error(test.es, res.es_1st_inv_diff))
    df_results_moni = df_results_moni.append({'p': p, 'q': q,'tr': tr,'RMSE_EURUSD': eur_usd_rmse,
                                              'RMSE_GBPUSD':gbp_usd_rmse,'RMSE_USDJPY':usd_jpy_rmse,
                                              'RMSE_GC':gc_rmse,'RMSE_NQ': nq_rmse, 
                                              'RMSE_ES': es_rmse}, ignore_index=True)
end = timer()
print(f' Total time taken to complete grid search in seconds:{(end - start)}')

starting grid search


In [ ]:
df_results_moni.sort_values(by = ['RMSE_EURUSD','RMSE_GBPUSD','RMSE_USDJPY',
                                        'RMSE_GC', 'RMSE_NQ', 'RMSE_ES'] )

In [ ]:
varmax_model = VARMAX(train_diff[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']],
               order=(1,2),trends = 'n').fit( disp=False)
result = varmax_model.forecast(steps = 15)

varmax_res = inverse_diff(train[['eur_usd', 'gbp_usd', 'usd_jpy','gc', 'nq', 'es']], result)
varmax_res